In [3]:
import psycopg2
import json


def recreate_tables(db_config: dict, create_tables_sql: str = "CREATE_TABLES.sql"):
    with open(create_tables_sql, "r") as file:
        create_tables_query = file.read()

    with psycopg2.connect(**db_config) as connection:
        cursor = connection.cursor()
        cursor.execute(create_tables_query)
        connection.commit()


def insert_table_json(db_config: dict, filepath: str, remove: list[str] = []):
    with open(filepath, "r") as file:
        data: list[dict] = json.load(file)
    for key in remove:
        for item in data:
            if key in item:
                del item[key]

    if data:
        columns = ", ".join(data[0].keys())
        data_json = json.dumps(data).replace("'", "''")
        table_name = filepath.split("/")[-1].split(".")[0]

        with psycopg2.connect(**db_config) as connection:
            cursor = connection.cursor()
            cursor.execute(f"""
                INSERT INTO {table_name} ({columns})
                SELECT {columns}
                FROM json_populate_recordset(
                    NULL::{table_name},
                    '{data_json}'
                )
                RETURNING *;
            """)
            connection.commit()


with open("database.json", "r") as file:
    database_config = json.load(file)

recreate_tables(database_config, "CREATE_TABLES.sql")

insert_table_json(
    database_config, "../mock_data/users.json", remove=["uid", "uid_uuid4"]
)
insert_table_json(database_config, "../mock_data/categories.json", remove=["cid"])
insert_table_json(database_config, "../mock_data/status.json", remove=["sid"])
insert_table_json(
    database_config,
    "../mock_data/transactions_active.json",
    remove=["tid", "tid_uuid4"],
)
insert_table_json(
    database_config, "../mock_data/items.json", remove=["iid", "iid_uuid4"]
)
insert_table_json(
    database_config,
    "../mock_data/transactions_archived.json",
    remove=["tid", "tid_uuid4"],
)

In [ ]:
"""
SELECT CURRENT_ROLE;
SELECT * FROM pg_settings WHERE setting ILIKE '%host%';
SELECT * FROM pg_settings WHERE name = 'port';


sudo systemctl start postgresql
sudo systemctl status postgresql
sudo systemctl stop postgresql

sudo -i -u postgres
psql
\password postgres


ALTER USER postgres PASSWORD 'mynewpassword'


Here's a more detailed breakdown:
1. Find the WSL IP Address:
Open your Ubuntu WSL terminal.
Run the command wsl hostname -I. This will output the IP address of your WSL instance. 
2. Configure PostgreSQL for Remote Access (if needed):
By default, PostgreSQL in WSL might only allow local connections. You may need to edit postgresql.conf and pg_hba.conf to allow connections from the WSL IP. 
Open postgresql.conf with sudo nano /etc/postgresql/<version>/main/postgresql.conf and change listen_addresses = 'localhost' to listen_addresses = '*' to allow connections from any IP. 
Open pg_hba.conf with sudo nano /etc/postgresql/<version>/main/pg_hba.conf and add a line like host all all <WSL_IP>/32 md5 to allow connections from your WSL IP (replace <WSL_IP> with the actual IP). 
Restart the PostgreSQL service after making these changes. 
3. Connect from Windows:
Use the WSL IP address you found earlier and the default PostgreSQL port (5432) in your connection string or configuration for your chosen PostgreSQL client tool (e.g., pgAdmin). 
Example:
If wsl hostname -I returns 172.20.37.128, and you're using pgAdmin, you would: 
Open pgAdmin.
Right-click on "Servers" and select "Register" -> "Server".
Name the server connection (e.g., "WSL PostgreSQL").
In the "Connection" tab, enter:
Host name/address: 172.20.37.128
Port: 5432
Username: The PostgreSQL username you want to use.
Password: The password for that user.
Click "Save".
"""

'\nHere\'s a more detailed breakdown:\n1. Find the WSL IP Address:\nOpen your Ubuntu WSL terminal.\nRun the command wsl hostname -I. This will output the IP address of your WSL instance. \n2. Configure PostgreSQL for Remote Access (if needed):\nBy default, PostgreSQL in WSL might only allow local connections. You may need to edit postgresql.conf and pg_hba.conf to allow connections from the WSL IP. \nOpen postgresql.conf with sudo nano /etc/postgresql/<version>/main/postgresql.conf and change listen_addresses = \'localhost\' to listen_addresses = \'*\' to allow connections from any IP. \nOpen pg_hba.conf with sudo nano /etc/postgresql/<version>/main/pg_hba.conf and add a line like host all all <WSL_IP>/32 md5 to allow connections from your WSL IP (replace <WSL_IP> with the actual IP). \nRestart the PostgreSQL service after making these changes. \n3. Connect from Windows:\nUse the WSL IP address you found earlier and the default PostgreSQL port (5432) in your connection string or config